In [12]:
import sqlite3

from pandas import DataFrame, read_csv, read_sql

import matplotlib.pyplot as plt
import pandas as pd
import sys
import matplotlib 
import numpy as np
import scipy.stats
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Enable inline plotting
%matplotlib inline

pd.set_option('display.max_columns', None)

import sys
sys.path.extend(['.', '..'])

from etl import cbs

In [13]:
logger.info(pcode.isnull().sum())

INFO:__main__:DONTUSE_OCHA_DCODE       0
DIST_CODE                0
DIST_NAME                0
DONTUSE_OCHA_VCODE       0
OCHA_VNAME               0
VDC_CODE                 0
VDC_NAME                 0
Unnamed: 7               0
Unnamed: 8            3989
Unnamed: 9            3989
dtype: int64


In [5]:
pcode = cbs.read_pcode('../data/' + cbs.etl.PCODE_NAME)
hh = cbs.add_dist_codes(cbs.read_hh('../data/' + cbs.etl.HH_NAME), pcode)

/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (22,23,57,58,69,70,72,73) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


INFO:etl.cbs:Adding district codes
INFO:etl.cbs:Mispelled districts:
INFO:etl.cbs:Sankhuwasabha
INFO:etl.cbs:Chitwan
INFO:etl.cbs:Tanahu
INFO:etl.cbs:Kavrepalanchok
INFO:etl.cbs:Makwanpur
INFO:etl.cbs:Fixed district mispellings, checking again:
INFO:etl.cbs:Mispelled districts:
INFO:etl.cbs:Unassigned districts in HH:
INFO:etl.cbs:Series([], Name: dist, dtype: object)


In [2]:
"""import"""

pcode = cbs.read_pcode('../data/' + cbs.etl.PCODE_NAME)
hh = cbs.add_dist_codes(cbs.read_hh('../data/' + cbs.etl.HH_NAME), pcode)
lookup = pd.read_csv('../data/nra_5w_lookup.csv')
nra = pd.read_csv('../data/public_vdc_assessment.csv')

pcode = pcode.rename(columns = {'DONTUSE_OCHA_VCODE' : 'vcode', 'DIST_CODE' : 'dist_code'})
lookup = lookup[[c for c in lookup.columns if not c.lower().startswith('unnamed')]]
lookup = lookup.rename(columns = {'HRRP Code' : 'hrrp_code', 'HLCIT Code' : 'hlcit_code', 'VDC' : 'vdc', 'Distrct' : 'district'})

/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (22,23,57,58,69,70,72,73) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


INFO:etl.cbs:Adding district codes
INFO:etl.cbs:Mispelled districts:
INFO:etl.cbs:Sankhuwasabha
INFO:etl.cbs:Chitwan
INFO:etl.cbs:Tanahu
INFO:etl.cbs:Kavrepalanchok
INFO:etl.cbs:Makwanpur
INFO:etl.cbs:Fixed district mispellings, checking again:
INFO:etl.cbs:Mispelled districts:
INFO:etl.cbs:Unassigned districts in HH:
INFO:etl.cbs:Series([], Name: dist, dtype: object)


In [39]:
"""check to see if there are the same number of VDCs in the pcode data as reported in hh"""

hh_g = hh[['vcode', 'dist_code']].drop_duplicates().groupby('dist_code').count()
pc_g = pcode[['vcode', 'dist_code']].drop_duplicates().groupby('dist_code').count()

res = pd.merge(hh_g, pc_g, left_index = True, right_index = True, how = 'left', suffixes = ['_hh', '_pc'])
res['diff'] = res['vcode_hh'] - res['vcode_pc']

import logging
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger(__name__)

logger.info(res)

INFO:__main__:             vcode_hh  vcode_pc  diff
dist_code                            
524 1 02 07        30        36    -6
524 1 02 09        27        34    -7
524 1 02 10        56        63    -7
524 1 03 11        31        35    -4
524 1 03 12        73        76    -3
524 1 03 13        56        56     0
524 2 04 20        54        54     0
524 2 04 21        55        55     0
524 2 04 22        52        52     0
524 2 05 23        79        79     0
524 2 05 24        90        90     0
524 2 05 25        26        42   -16
524 2 05 26         6        18   -12
524 2 05 27        11        59   -48
524 2 05 28        62        62     0
524 2 05 29        18        18     0
524 2 05 30        50        50     0
524 2 06 31        44        45    -1
524 2 06 35        16        39   -23
524 3 07 36        67        67     0
524 3 07 37        40        61   -21
524 3 07 39        34        45   -11
524 3 07 40        42        47    -5
524 3 07 41        57        62    -

In [ ]:
hh.to_csv('../data/coded_unique_hh.csv')

In [5]:
"""create pcode hh merge, check to see if all the VDC nums in hh are included in pcode"""
p_sm = pcode[['vcode', 'dist_code', 'VDC_NAME', 'VDC_CODE', 'DIST_NAME']]
p_sm['mrg'] = p_sm['dist_code'] + p_sm.vcode.apply(lambda x : str(int(x.split('-')[-1])))

hh_sm = hh[['dist_code', 'vdcmun', 'vcode', 'dist']].drop_duplicates()
hh_sm['mrg'] = hh_sm['dist_code'] + hh_sm['vdcmun'].apply(str)

hh_pc_mrg = pd.merge(hh_sm, p_sm, left_on = 'mrg', right_on = 'mrg', how = 'left', suffixes = ['_hh', '_pc'])

PRIORITY = ['Bhaktapur',
            'Kathmandu',
            'Lalitpur',
            'Okhaldhunga',
            'Sindhuli',
            'Ramechhap',
            'Dolakha',
            'Sindhupalchok',
            'Kabhrepalanchok',
            'Nuwakot',
            'Rasuwa',
            'Dhading',
            'Makawanpur',
            'Gorkha']

hh_pc_mrg['priority'] = hh_pc_mrg.apply(lambda x : True if x['DIST_NAME'] in PRIORITY else False, axis = 1)

hh_pc_mrg.isnull().sum()

/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


dist_code_hh    0
vdcmun          0
vcode_hh        0
dist            0
mrg             0
vcode_pc        0
dist_code_pc    0
VDC_NAME        0
VDC_CODE        0
DIST_NAME       0
priority        0
dtype: int64

In [15]:
hh[['dist', 'vcode']]

,dist,vcode
0,Dhankuta,Ahale
1,Dhankuta,Ahale
2,Dhankuta,Ahale
3,Dhankuta,Ahale
4,Dhankuta,Ahale
5,Dhankuta,Ahale
6,Dhankuta,Ahale
7,Dhankuta,Ahale
8,Dhankuta,Ahale
9,Dhankuta,Ahale


In [6]:
"""check to see if there are the names of VDCs approximately match"""

hh_pc_mrg['sim'] = hh_pc_mrg.apply(lambda x : fuzz.partial_ratio(x['vcode_hh'], x['VDC_NAME']), axis = 1)

pd.set_option('display.max_rows', None)
hh_pc_mrg[['VDC_NAME', 'vcode_hh']][hh_pc_mrg['sim'] < 70]
pd.set_option('display.max_rows', 100)


In [7]:
if 'dist_code_pc' in hh_pc_mrg.columns:
    hh_pc_mrg = hh_pc_mrg.drop(['dist_code_pc'], axis = 1)

hh_pc_mrg = hh_pc_mrg.rename(columns = {'VDC_CODE' : 'hclit_code_pc', 'DIST_NAME' : 'dist_name_pc', 'VDC_NAME' : 'vdc_name_pc',
                     'vdcode_hh' : 'vdc_name_hh', 'mrg' : 'vdc_dist_mrg_hh', 'dist' : 'dist_hh'})

# hh_pc_mrg['concat'] = hh_pc_mrg['dist_hh'] + hh_pc_mrg['vcode_hh']

hh_pc_mrg

,dist_code_hh,vdcmun,vcode_hh,dist_hh,vdc_dist_mrg_hh,vcode_pc,vdc_name_pc,hclit_code_pc,dist_name_pc,priority,sim
0,524 1 02 07,1,Ahale,Dhankuta,524 1 02 071,E-KOS-07-001,Aahale,524 1 02 07 5 002,Dhankuta,False,80
1,524 1 02 07,2,Ankhisalla,Dhankuta,524 1 02 072,E-KOS-07-002,Aakhi Salla,524 1 02 07 5 016,Dhankuta,False,70
2,524 1 02 07,3,Arkhaule Jitpur,Dhankuta,524 1 02 073,E-KOS-07-003,Jeetpur,524 1 02 07 5 022,Dhankuta,False,77
3,524 1 02 07,4,Basantatar,Dhankuta,524 1 02 074,E-KOS-07-004,Bashantatar,524 1 02 07 5 012,Dhankuta,False,90
4,524 1 02 07,7,Bodhe,Dhankuta,524 1 02 077,E-KOS-07-007,Bodhe,524 1 02 07 5 007,Dhankuta,False,100
5,524 1 02 07,8,Chha Nambar Budhabare,Dhankuta,524 1 02 078,E-KOS-07-008,Budhbare,524 1 02 07 5 013,Dhankuta,False,88
6,524 1 02 07,9,Budi Morang,Dhankuta,524 1 02 079,E-KOS-07-009,Bhudhemorang,524 1 02 07 5 004,Dhankuta,False,64
7,524 1 02 07,10,Chanuwa,Dhankuta,524 1 02 0710,E-KOS-07-010,Chanuwa,524 1 02 07 5 025,Dhankuta,False,100
8,524 1 02 07,11,Chhintang,Dhankuta,524 1 02 0711,E-KOS-07-011,Chintang,524 1 02 07 5 035,Dhankuta,False,88
9,524 1 02 07,12,Chungwang,Dhankuta,524 1 02 0712,E-KOS-07-012,Chungbang,524 1 02 07 5 033,Dhankuta,False,89


In [30]:
#check to see that all of lookup's VDC codes are in vhh_pc_mrg

hh_pc_look_mrg = pd.merge(lookup, hh_pc_mrg, left_on = 'hlcit_code', right_on = 'hclit_code_pc', how = 'left')
# t['sim'] = t.apply(lambda x : fuzz.partial_ratio(str(x['vcode_hh']), str(x['vdc_name_pc']), axis = 1))

# hh_pc_look_mrg[['dist_hh', 'vcode_hh', 'hlcit_code', 'vdc_name_pc', 'dist_name_pc', 'hclit_code_pc']]

nra['concat'] = nra['district'] + nra['vdc_municipality']
nra = pd.merge(nra, hh_pc_look_mrg, on = 'concat', how = 'left', suffixes = ['_nra', '_look'])

nra['dist_hh_look'].isnull().sum()

dist_hh_look    15
dist_hh_look    15
dist_hh_look    15
dtype: int64

In [37]:
nra[nra['vcode_pc'].isnull()]

,district,vdc_municipality,total_HH,no_hh_with_grant_agreement,no_hh_with_first_installment,site_layout,houses_not_varified_for_second_installment,houses_varified_for_second_installment,houses_not_able_to_varified_for_sec_instal,remaining_for_second_install,houses_not_varified_for_third_installment,houses_varified_for_third_installment,houses_not_able_to_verified_for_third_instal,remaining_for_third_install,construction_completed,complainace,complaince_resolved,self_completed_houses,engineer_name,engineer_contact_no,remark,uid,data_entered_date,concat,hrrp_code_nra,District_nra,vdc_nra,hlcit_code_nra,dist_code_hh_nra,vdcmun_nra,vcode_hh_nra,dist_hh_nra,vdc_dist_mrg_hh_nra,vcode_pc_nra,vdc_name_pc_nra,hclit_code_pc_nra,dist_name_pc_nra,priority_nra,sim_nra,hrrp_code_look,District_look,vdc_look,hlcit_code_look,dist_code_hh_look,vdcmun_look,vcode_hh_look,dist_hh_look,vdc_dist_mrg_hh_look,vcode_pc_look,vdc_name_pc_look,hclit_code_pc_look,dist_name_pc_look,priority_look,sim_look,hrrp_code_nra,District_nra,vdc_nra,hlcit_code_nra,dist_code_hh_nra,vdcmun_nra,vcode_hh_nra,dist_hh_nra,vdc_dist_mrg_hh_nra,vcode_pc_nra,vdc_name_pc_nra,hclit_code_pc_nra,dist_name_pc_nra,priority_nra,sim_nra,hrrp_code_look,District_look,vdc_look,hlcit_code_look,dist_code_hh_look,vdcmun_look,vcode_hh_look,dist_hh_look,vdc_dist_mrg_hh_look,vcode_pc_look,vdc_name_pc_look,hclit_code_pc_look,dist_name_pc_look,priority_look,sim_look,hrrp_code_nra,District_nra,vdc_nra,hlcit_code_nra,dist_code_hh_nra,vdcmun_nra,vcode_hh_nra,dist_hh_nra,vdc_dist_mrg_hh_nra,vcode_pc_nra,vdc_name_pc_nra,hclit_code_pc_nra,dist_name_pc_nra,priority_nra,sim_nra,dist_code_hh_look,vdcmun_look,vcode_hh_look,dist_hh_look,vdc_dist_mrg_hh_look,vcode_pc_look,vdc_name_pc_look,hclit_code_pc_look,dist_name_pc_look,priority_look,sim_look,hrrp_code_look,District_look,vdc_look,hlcit_code_look,hrrp_code,District,vdc,hlcit_code,dist_code_hh,vdcmun,vcode_hh,dist_hh,vdc_dist_mrg_hh,vcode_pc,vdc_name_pc,hclit_code_pc,dist_name_pc,priority,sim
58,LALITPUR,Godawari,3256,2476.0,1644.0,142.0,77.0,31.0,10.0,29.0,17.0,NaN,5.0,NaN,NaN,824.0,626.0,NaN,Bimala Dangol,9841508890,NaN,14,2017-09-13 11:52:43+00,LALITPURGodawari,524 2 05 25 3 019,LALITPUR,Godawari,524 2 05 25 5 023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,524 2 05 25 3 019,LALITPUR,Godawari,524 2 05 25 5 023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,524 2 05 25 3 019,LALITPUR,Godawari,524 2 05 25 5 023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,524 2 05 25 3 019,LALITPUR,Godawari,524 2 05 25 5 023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,524 2 05 25 3 019,LALITPUR,Godawari,524 2 05 25 5 023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,524 2 05 25 3 019,LALITPUR,Godawari,524 2 05 25 5 023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,LALITPUR,KaryaBinayak,3305,2461.0,2095.0,20.0,10.0,5.0,14.0,NaN,10.0,NaN,NaN,10.0,10.0,1320.0,1172.0,NaN,Tilasmi Subba,"9849224276, 9851223635,",RCC - ( nos Most cases Area Exceeded between 4...,14,2017-09-13 11:58:37+00,LALITPURKaryaBinayak,524 2 05 25 3 007,LALITPUR,KaryaBinayak,524 2 05 25 5 020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,524 2 05 25 3 007,LALITPUR,KaryaBinayak,524 2 05 25 5 020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,524 2 05 25 3 007,LALITPUR,KaryaBinayak,524 2 05 25 5 020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,524 2 05 25 3 007,LALITPUR,KaryaBinayak,524 2 05 25 5 020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,524 2 05 25 3 007,LALITPUR,KaryaBinayak,524 2 05 25 5 020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,524 2 05 25 3 007,LALITPUR,KaryaBinayak,524 2 05 25 5 020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,LALITPUR,Bajrabarahi,4023,3098.0,2418.0,141.0,138.0,38.0,7.0,82.0,NaN,10.0,NaN,NaN,4.0,829.0,433.0,5.0,Sunita Kutu,9841963582,6- RCC ( Exceeded Area) & 1 Load Bearing Not ...,14,2017-09-13 12:32:21+00,LALITPURBajrabarahi,524 2 05 25 3 009,LALITPUR,Bajrabarah

In [23]:
hh_pc_look_mrg = pd.merge(hh_pc_mrg, lookup, left_on = 'hclit_code_pc', right_on = 'hlcit_code', how = 'left', suffixes = ['_mrg', '_look'])

nra['concat'] = nra['district'] + nra['vdc_municipality']

nra = pd.merge(nra, hh_pc_look_mrg, on = 'concat', how = 'left', suffixes = ['_nra', '_look'])

In [ ]:
nra.iloc[612]['district'] + nra.iloc[612]['vdc_municipality'] == lookup.iloc[195]['concat']

In [ ]:
len(nra) - nra['VDC_CODE'].isnull().sum()

In [ ]:
nra['concat'] = nra['district'] + nra['vdc_municipality']

m = pd.merge(nra, lookup, on = 'concat', how = 'left')

In [ ]:
m[m['hlcit_code'].isnull()]

In [16]:
from etl.joins import Dataset

d = Dataset()
d = d.get_nra()


/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2862: DtypeWarning: Columns (22,23,57,58,69,70,72,73) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


INFO:etl.joins:Diff in counts from CBS data, OCHA pcode:
INFO:etl.joins:             vcode_hh  vcode_pc  diff
dist_code                            
524 1 02 07        30        36    -6
524 1 02 09        27        34    -7
524 1 02 10        56        63    -7
524 1 03 11        31        35    -4
524 1 03 12        73        76    -3
524 1 03 13        56        56     0
524 2 04 20        54        54     0
524 2 04 21        55        55     0
524 2 04 22        52        52     0
524 2 05 23        79        79     0
524 2 05 24        90        90     0
524 2 05 25        26        42   -16
524 2 05 26         6        18   -12
524 2 05 27        11        59   -48
524 2 05 28        62        62     0
524 2 05 29        18        18     0
524 2 05 30        50        50     0
524 2 06 31        44        45    -1
524 2 06 35        16        39   -23
524 3 07 36        67        67     0
524 3 07 37        40        61   -21
524 3 07 39        34        45   -11
524 3 07 40     

../etl/joins.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  p_sm['mrg'] = p_sm['dist_code'] + p_sm.vcode.apply(lambda x: str(int(x.split('-')[-1])))


INFO:etl.joins:Counts of nulls in merge of CBS, OCHA pcodes
INFO:etl.joins:dist_code_hh    0
vdcmun          0
vcode_hh        0
dist            0
mrg             0
vcode_pc        0
dist_code_pc    0
VDC_NAME        0
VDC_CODE        0
DIST_NAME       0
priority        0
dtype: int64
INFO:etl.joins:
INFO:etl.joins:


/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (22,23,57,58,69,70,72,73) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


INFO:etl.joins:Diff in counts from CBS data, OCHA pcode:
INFO:etl.joins:             vcode_hh  vcode_pc  diff
dist_code                            
524 1 02 07        30        36    -6
524 1 02 09        27        34    -7
524 1 02 10        56        63    -7
524 1 03 11        31        35    -4
524 1 03 12        73        76    -3
524 1 03 13        56        56     0
524 2 04 20        54        54     0
524 2 04 21        55        55     0
524 2 04 22        52        52     0
524 2 05 23        79        79     0
524 2 05 24        90        90     0
524 2 05 25        26        42   -16
524 2 05 26         6        18   -12
524 2 05 27        11        59   -48
524 2 05 28        62        62     0
524 2 05 29        18        18     0
524 2 05 30        50        50     0
524 2 06 31        44        45    -1
524 2 06 35        16        39   -23
524 3 07 36        67        67     0
524 3 07 37        40        61   -21
524 3 07 39        34        45   -11
524 3 07 40     

In [17]:
d

,district,vdc_municipality,total_HH,no_hh_with_grant_agreement,no_hh_with_first_installment,site_layout,houses_not_varified_for_second_installment,houses_varified_for_second_installment,houses_not_able_to_varified_for_sec_instal,remaining_for_second_install,houses_not_varified_for_third_installment,houses_varified_for_third_installment,houses_not_able_to_verified_for_third_instal,remaining_for_third_install,construction_completed,complainace,complaince_resolved,self_completed_houses,engineer_name,engineer_contact_no,remark,uid,data_entered_date,concat,hrrp_code,District,vdc,hlcit_code,dist_code_hh,vdcmun,vcode_hh,dist_hh,vdc_dist_mrg_hh,vcode_pc,vdc_name_pc,hclit_code_pc,dist_name_pc,priority
0,KABHREPALANCHOK,Chauri Pokhari,678,660.0,660.0,36.0,36.0,36.0,NaN,36.0,36.0,36.0,NaN,36.0,36.0,313.0,72.0,16.0,Dinesh Raj Bhattrai,9848787762,NaN,13,2017-07-02 19:21:05+00,KABHREPALANCHOKChauri Pokhari,524 2 05 24 5 068,KABHREPALANCHOK,Chauri Pokhari,524 2 05 24 5 042,524 2 05 24,68.0,Pokhari Chauri,Kabhrepalanchok,524 2 05 2468,C-BAG-24-068,Chauri Pokhari,524 2 05 24 5 042,Kabhrepalanchok,True
1,KABHREPALANCHOK,Simthali,441,407.0,407.0,90.0,88.0,88.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,pradip pokharel,9851213857,NaN,13,2017-07-03 16:50:33+00,KABHREPALANCHOKSimthali,524 2 05 24 5 085,KABHREPALANCHOK,Simthali,524 2 05 24 5 087,524 2 05 24,85.0,Simthali,Kabhrepalanchok,524 2 05 2485,C-BAG-24-085,Simthali,524 2 05 24 5 087,Kabhrepalanchok,True
2,KABHREPALANCHOK,Milche,436,435.0,435.0,32.0,24.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,3.0,Animesh Nepal,9842084633,NaN,13,2017-07-03 20:38:06+00,KABHREPALANCHOKMilche,524 2 05 24 5 060,KABHREPALANCHOK,Milche,524 2 05 24 5 045,524 2 05 24,60.0,Milche,Kabhrepalanchok,524 2 05 2460,C-BAG-24-060,Milche,524 2 05 24 5 045,Kabhrepalanchok,True
3,GORKHA,Laprak,573,573.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,2017-07-04 15:19:10+00,GORKHALaprak,524 3 07 36 5 037,GORKHA,Laprak,524 3 07 36 5 047,524 3 07 36,37.0,Laprak,Gorkha,524 3 07 3637,W-GAN-36-037,Laprak,524 3 07 36 5 047,Gorkha,True
4,GORKHA,Lho,249,246.0,209.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,"Due to Winter Season for last few months, the ...",11,2017-07-04 15:19:42+00,GORKHALho,524 3 07 36 5 039,GORKHA,Lho,524 3 07 36 5 040,524 3 07 36,39.0,Lho,Gorkha,524 3 07 3639,W-GAN-36-039,Lho,524 3 07 36 5 040,Gorkha,True
5,GORKHA,Samagaun,224,220.0,207.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,Due to winter season the technicians were depl...,11,2017-07-04 15:20:21+00,GORKHASamagaun,524 3 07 36 5 052,GORKHA,Samagaun,524 3 07 36 5 042,524 3 07 36,52.0,Samagaun,Gorkha,524 3 07 3652,W-GAN-36-052,Samagaun,524 3 07 36 5 042,Gorkha,True
6,NUWAKOT,Likhu,707,683.0,683.0,156.0,150.0,174.0,NaN,49.0,NaN,NaN,NaN,NaN,NaN,35.0,34.0,26.0,Sulabh Aryal,9841155104,NaN,16,2017-07-05 16:04:20+00,NUWAKOTLikhu,524 2 05 28 5 037,NUWAKOT,Likhu,524 2 05 28 5 017,524 2 05 28,37.0,Likhu,Nuwakot,524 2 05 2837,C-BAG-28-037,Likhu,524 2 05 28 5 017,Nuwakot,True
7,NUWAKOT,Samudratar,500,486.0,486.0,130.0,130.0,122.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,68.0,66.0,34.0,Santosh Giri,9849738227,NaN,16,2017-07-05 16:40:53+00,NUWAKOTSamudratar,524 2 05 28 5 050,NUWAKOT,Samudratar,524 2 05 28 5 032,524 2 05 28,50.0,Samundratar,Nuwakot,524 2 05 2850,C-BAG-28-050,Samudratar,524 2 05 28 5 032,Nuwakot,True
8,NUWAKOT,Rautbesi,796,783.0,783.0,86.0,60.0,88.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,115.0,110.0,108.0,Subhas Basnet,9813054618,NaN,16,2017-07-05 16:50:53+00,NUWAKOTRautbesi,524 2 05 28 5 046,NUWAKOT,Rautbesi,524 2 05 28 5 025,524 2 05 28,46.0,Rautbesi,Nuwakot,524 2 05 2846,C-BAG-28-046,Rautbesi,524 2 05 28 5 025,Nuwakot,True
9,NUWAKOT,Lachyang,1094,1060.0,1060.0,44.0,186.0,123.0,33.0,30.0,NaN,NaN,NaN,NaN,NaN,68.0,68.0,30.0,Atik Raj Pathak,9845294313,3 houses are of trusses,16,2017-07-05 16:52:25+00,NUWAKOTLachyang,524 2 05 28 5 036,NUWAKOT,Lachyang,524 2 05 28 5 042,524 2 05 28,36.0,Lachyang,Nuwakot,524 2 05 2836,C-BAG-28-036,Lach